In [ ]:
# Load all files from the 'examples' directory
llm_context_manager = LLMContextManager.load_from_directory(dir_path='examples')

In [ ]:
# Print out one of the loaded sections
print(f"Contents of the 'example_section' section: {llm_context_manager.sections.get('doc1', 'Section not found.')}")
print(f"llm_prompt.get_section_keys(): {llm_context_manager.get_section_keys()}")